In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
def get_edges(line): # grafo no dirigido, sin bucles.
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 < n2:
         return (n1,n2)
    elif n1 > n2:
         return (n2,n1)
    else:
        pass #n1 == n2

In [19]:
def degree(sc, filename):
    nodes = sc.textFile(filename).\
        map(get_edges).\
        filter(lambda x: x is not None).\
        distinct().\
        flatMap(lambda x: [x[0],x[1]])
    print(nodes.collect())
    print(nodes.countByKey())
        

In [20]:
degree(sc, "g0.txt")

['A', 'B', 'B', 'C', 'A', 'C']
defaultdict(<class 'int'>, {'A': 2, 'B': 2, 'C': 2})


In [21]:
degree(sc, "g3.txt")

['A', 'B', 'A', 'D', 'A', 'F', 'A', 'C']
defaultdict(<class 'int'>, {'A': 4, 'B': 1, 'D': 1, 'F': 1, 'C': 1})


In [67]:
def get_rdd_distict_edges(sc, filename):
    return sc.textFile(filename).\
        map(get_edges).\
        filter(lambda x: x is not None).\
        distinct()

def degree(sc, filename):
    nodes = get_rdd_distict_edges(sc, filename).\
        flatMap(lambda x: [x[0],x[1]])
    print(nodes.countByKey())
    

In [27]:
degree(sc, "g0.txt")

defaultdict(<class 'int'>, {'A': 2, 'B': 2, 'C': 2})


In [28]:
degree(sc, "g3.txt")

defaultdict(<class 'int'>, {'A': 4, 'B': 1, 'D': 1, 'F': 1, 'C': 1})


In [38]:
def adjacents(sc, filename):
    nodes = get_rdd_distict_edges(sc, filename)
    adj = nodes.groupByKey().collect()
    print(adj)
    for node in adj:
        print(node[0], list(node[1]))

In [39]:
adjacents(sc, "g0.txt")

[('A', <pyspark.resultiterable.ResultIterable object at 0x7f1d78f0f340>), ('B', <pyspark.resultiterable.ResultIterable object at 0x7f1d78f0bd30>)]
A ['B', 'C']
B ['C']


In [68]:
def get_edges_rep(line):
    edge = line.strip().split(',')
    n1 = edge[0]
    n2 = edge[1]
    if n1 != n2:
         return [(n1,n2), (n2,n1)]
    else:
        return [(n1,n2)] #n1 == n2

In [69]:
def adjacents_2(sc, filename):
    edges = sc.textFile(filename).\
        flatMap(get_edges_rep).\
        filter(lambda x: x is not None).\
        distinct()
    adj = edges.groupByKey().collect()
    for node in adj:
        print(node[0], list(node[1]))

In [70]:
adjacents_2(sc, "g0.txt")

C ['B', 'A']
A ['B', 'C']
B ['A', 'C']


In [71]:
adjacents(sc, "g3.txt")

[('A', <pyspark.resultiterable.ResultIterable object at 0x7f1d791b9e50>)]
A ['B', 'D', 'F', 'C']


In [73]:
!cat g3.txt

A,B
A,C
A,D
A,F
F,F
F,A


In [72]:
adjacents_2(sc, "g3.txt")

C ['A']
A ['B', 'D', 'F', 'C']
B ['A']
D ['A']
F ['A', 'F']


In [89]:
def same_adjacents(sc, filename):
    edges = sc.textFile(filename).\
        flatMap(get_edges_rep).\
        filter(lambda x: x is not None).\
        distinct()
    adj = edges.groupByKey()
    # 1. Necesitamos que la lista de vecinos esté ordenada para
    # así poder comparar con otras listas de vecinos;
    # 2. Para que pueda ser clave, tenemos que convertir la lista en tupla.
    inv_adj = adj.map( lambda x: (tuple( sorted( x[1] )), x[0]) )
    for adj, nodes in inv_adj.groupByKey().filter(lambda x: len(x[1])>1 ).collect():
        print(adj, list(nodes))
        

In [90]:
same_adjacents(sc, "g3.txt")

('A',) ['C', 'B', 'D']
